In [1]:
import torch as T
from torch import nn as nn
from torch.nn import functional as F
from embedding import CharacterConvEmbedding
from attention import BasicAttention, BiAttention, CoAttention
from network import EmbedText, Encoder, DynamicPointingDecoder

In [2]:
char_args = {'vocab_size': 70, 'embedding_dim': 8, 'padding_idx':10}
char_conv_args = {'char_embed_dim':8, 'embedding_dim':100, 'kernel_sizes':[2, 3, 4 ,5], 'num_filters':50}
word_args = {'vocab_size':40000, 'embedding_dim':100, 'padding_idx':10}

c_char = T.zeros([32, 600, 50], dtype=T.long)
c_word = T.zeros([32, 600], dtype=T.long)
q_char = T.zeros([32, 50, 50], dtype=T.long)
q_word = T.zeros([32, 50], dtype=T.long)

In [3]:
embedtext = EmbedText(word_args, char_args, shared=False)
qw, cw, qc, cc = embedtext(q_word, c_word, q_char, c_char)
qw = qw.cuda()
cw = cw.cuda()
qc = qc.cuda()
cc = cc.cuda()

print (qw.type(), cw.type(), qc.type(), cc.type())

torch.cuda.FloatTensor torch.cuda.FloatTensor torch.cuda.FloatTensor torch.cuda.FloatTensor


In [4]:
encoder = Encoder(word_args, char_conv_args, hidden_dim=200, shared=True, use_cuda=True)

q_lstm, c_lstm = encoder(qw, cw, qc, cc)

print (q_lstm.type(), c_lstm.type())
print (q_lstm.shape, c_lstm.shape)

torch.cuda.FloatTensor torch.cuda.FloatTensor
torch.Size([32, 50, 400]) torch.Size([32, 600, 400])


In [5]:
i = CoAttention(200, use_cuda=True)
o = i(q_lstm, c_lstm)

o.shape

torch.Size([32, 600, 800])

In [6]:
dn = DynamicPointingDecoder(hidden_dim=400, use_cuda=True)

In [7]:
dn(o)

(tensor([ 599,  599,  599,  599,  599,  599,  599,  599,  599,  599,
          599,  599,  599,  599,  599,  599,  599,  599,  599,  599,
          599,  599,  599,  599,  599,  599,  599,  599,  599,  599,
          599,  599], device='cuda:0'),
 tensor([ 599,  599,  599,  599,  599,  599,  599,  599,  599,  599,
          599,  599,  599,  599,  599,  599,  599,  599,  599,  599,
          599,  599,  599,  599,  599,  599,  599,  599,  599,  599,
          599,  599], device='cuda:0'),
 [[tensor(1.00000e-02 *
          [[ 5.5623,  5.6655,  5.6609,  ...,  6.1996,  6.5787,  7.5521],
           [ 5.8287,  5.8563,  5.8453,  ...,  6.4582,  6.9495,  7.7475],
           [ 5.9363,  6.0408,  6.1307,  ...,  6.6884,  7.1567,  8.0594],
           ...,
           [ 6.4139,  6.6795,  6.9410,  ...,  7.1870,  7.3650,  8.2088],
           [ 6.5386,  6.6904,  6.9493,  ...,  7.2008,  7.3920,  8.0821],
           [ 6.5748,  6.8221,  6.9110,  ...,  7.2317,  7.3989,  8.0391]], device='cuda:0'),
   tensor